# 07 - Interactive Queries

The examples in the previous notebooks, we learned how to run TEEHR queries by manually defining all components of the query.  Now that we know how the queries work, we will add interactive Panel widgets and a filter-building wrapper to make it even easier to query the portion of data you are interested in.

Note that for Notebooks 07-09, the file `postevent_dashboard_utils.py` must be imported.  It provides additional functionality needed for generating dashboards for the post-event evaluation examples. Some of this functionality may become part of the TEEHR package in the near future.


In [ ]:
import postevent_dashboard_utils as du
from pathlib import Path
import pandas as pd
import numpy as np
import geopandas as gpd
import panel as pn
import colorcet as cc
import hvplot
import hvplot.pandas
pn.extension()

### Define the set parquet files for each specific study/scenario

For this example, we use dictionaries to define multiple sets of parquet files needed to evaluate operational NWM short- and medium-range streamflow and precipitation forecasts.  As described in previous notebooks, these files define the source of verifying data (primary_filepath), data to evaluate (secondary_filepath), as well as the necessary geometry, crosswalk, and attributes.

In [ ]:
# overall study directory
CACHE_DIR = Path(Path.home(), "shared", "teehr-workshop", "post-event-example")

# medium range streamflow forecast evaluation files 
MRF_streamflow = dict(
    scenario_name="medium_range",
    variable="streamflow",
    primary_filepath=Path(CACHE_DIR, "timeseries", "usgs", "*.parquet"),
    secondary_filepath=Path(CACHE_DIR, "timeseries", "medium_range_mem1", "*.parquet"),
    crosswalk_filepath=Path(CACHE_DIR, "geo", "usgs_nwm22_crosswalk.parquet"),
    geometry_filepath=Path(CACHE_DIR, "geo", "usgs_geometry.parquet")
)
# medium range precip forecast evaluation files
MRF_forcing = dict(
    scenario_name="medium_range",
    variable="precipitation",    
    primary_filepath=Path(CACHE_DIR, "timeseries", "forcing_analysis_assim", "*.parquet"),
    secondary_filepath=Path(CACHE_DIR, "timeseries", "forcing_medium_range", "*.parquet"),
    crosswalk_filepath=Path(CACHE_DIR, "geo", "huc10_huc10_crosswalk.parquet"),
    geometry_filepath=Path(CACHE_DIR, "geo", "huc10_geometry.parquet"),
)
# short range streamflow forecast evaluation files 
SRF_streamflow = dict(
    scenario_name="short_range",
    variable="streamflow",
    primary_filepath=MRF_streamflow["primary_filepath"],
    secondary_filepath=Path(CACHE_DIR, "timeseries", "short_range", "*.parquet"),
    crosswalk_filepath=MRF_streamflow["crosswalk_filepath"],
    geometry_filepath=MRF_streamflow["geometry_filepath"],
)
# medium range precip forecast evaluation files
SRF_forcing = dict(
    scenario_name="short_range",
    variable="precipitation",    
    primary_filepath=MRF_forcing["primary_filepath"],
    secondary_filepath=Path(CACHE_DIR, "timeseries", "forcing_short_range", "*.parquet"),
    crosswalk_filepath=MRF_forcing["crosswalk_filepath"],
    geometry_filepath=MRF_forcing["geometry_filepath"],
)
attribute_paths = dict(
    usgs_upstream_area=Path(CACHE_DIR, "geo", "usgs_attr_upstream_area.parquet"),
    usgs_ecoregions=Path(CACHE_DIR, "geo", "usgs_attr_ecoregions.parquet"),
    usgs_stream_order=Path(CACHE_DIR, "geo", "usgs_attr_stream_order.parquet"),
    usgs_huc_crosswalk=Path(CACHE_DIR, "geo", "usgs_huc12_crosswalk.parquet"),
)

# put the scenarios in a list for widget purposes
scenario_definitions = [MRF_streamflow, MRF_forcing, SRF_streamflow, SRF_forcing]

## general units ('english' or 'metric') to show in visualization
viz_units = "metric"

### First select the scenario and variable
Set up dropdown menus to select the specific scenario to evaluate based on the scenario name and variable

In [ ]:
scenario_selector = du.get_scenario_selector(scenario_name_list=sorted(du.get_scenario_names(scenario_definitions)))  
variable_selector = du.get_variable_selector(variable_list=du.get_scenario_variables(scenario_definitions))   
pn.Row(scenario_selector, variable_selector)

### Then select the region, time period, characteristics and metrics of interest

Set up dropdowns and sliders to get the list or range of options and enable you to easily select a particular subset of data for metric calculations.

In [ ]:
scenario = du.get_scenario(scenario_definitions, scenario_selector.value, variable_selector.value)
widgets = du.get_filter_widgets(scenario=scenario, include_widgets=[
    'value_time','reference_time','lead_time','huc2','threshold','stream_order','metrics'])

pn.Row(
    pn.Column(widgets['huc2'], widgets['stream_order'], widgets['threshold'], widgets['metrics']),
    pn.Spacer(width=50),    
    pn.Column(     
        pn.Spacer(height=10), widgets['value_time'],
        pn.Spacer(height=10), widgets['reference_time'],
        pn.Spacer(height=5), widgets['lead_time'],
    )
)

### Make selections

Using the menus and sliders above, make selections to evaluate a particular subset of the data.  Many more filters are possible here, these are just a few examples we anticipate may be commonly needed.

### Run the query
Run the cell below to execute a wrapper which builds the filter portion of the TEEHR query and then executes the query.  Experiment with the filter widget selections to see how the run time and query output change.  

In [ ]:
%%time
gdf = du.run_teehr_query(
    query_type="metrics",
    scenario=scenario,
    huc_id=widgets['huc2'].value,
    order_limit=widgets['stream_order'].value,
    value_time_start=widgets['value_time'][1].value_start,    
    value_time_end=widgets['value_time'][1].value_end,    
    reference_time_start=widgets['reference_time'][1].value_start,    
    reference_time_end=widgets['reference_time'][1].value_end,
    group_by=['primary_location_id'],
    order_by=['primary_location_id'],
    value_min=widgets['threshold'].value,    
    include_metrics=widgets['metrics'].value,
    attribute_paths=attribute_paths,
    return_query=False,
)
display(gdf.head())

### Create a simple map of results

In [ ]:
gdf = gdf.to_crs("EPSG:3857")
gdf['relative_peak_error'] = gdf['max_value_delta']/gdf['primary_maximum']
gdf.hvplot.points(c='relative_peak_error', cmap=cc.CET_D1A[::-1], clim=(-1,1), width=600, height=400,
                             title="Relative Peak Error", size=10, xaxis = None, yaxis = None, tiles='CartoLight', 
                             hover_cols=['primary_location_id', 'primary_maximum'], cnorm='linear')


### Choose a gage to explore more closely
Using the zoom and hover features on the map, select a gage and manually enter the ID in the cell below. (Gage selection by clicking on points in the map will be demonstrated in notebook 09).

In [ ]:
usgs_id = 'usgs-11451715'

Using the wrapper again, build and run a TEEHR timeseries query to extract the streamflow timeseries.

In [ ]:
flow_df = du.run_teehr_query(
    query_type="timeseries",
    scenario=scenario,
    location_id=usgs_id,
    order_limit=widgets['stream_order'].value,
    value_time_start=widgets['value_time'][1].value_start,    
    value_time_end=widgets['value_time'][1].value_end,    
    reference_time_start=widgets['reference_time'][1].value_start,    
    reference_time_end=widgets['reference_time'][1].value_end,
    value_min=widgets['threshold'].value,    
    attribute_paths=attribute_paths,
    return_query=False,
    include_geometry=False,
)
display(flow_df.head())

### Plot the timeseries by reference time
Loop through the time series, sequentially adding an additional curve to the holoviews overlay `hydrographs` and further overlay with the observed data

In [ ]:
flow_df = flow_df.rename(columns={'primary_value':'obs_flow','secondary_value':'fcst_flow'})
ref_times = sorted(flow_df['reference_time'].unique())
cmap = cc.rainbow[::-1]
cstep = int(np.floor(len(cmap) / len(ref_times)))

flow_obs = flow_df.hvplot(x='value_time', y='obs_flow', label = 'observed', ylabel='Flow (cms)', color = 'black', line_width=4)   
hydrographs = flow_obs
for t, time in enumerate(ref_times):
    df_t = flow_df[flow_df['reference_time'] == time]
    ci = cmap[cstep * t]
    fcst = df_t.hvplot(x='value_time', y='fcst_flow', color = [ci])
    hydrographs = hydrographs * fcst

hydrographs * flow_obs

### Now let's get the precipitation analysis and forecasts as well

For basic comparison purposes, find and extract the mean areal precipitation time series for the HUC10 containing the above gage.  Eventually we will want to calculate the effective mean areal timeseries for the entire upstream drainage basin associated with a gage or other point.

In [ ]:
# read the usgs-huc crosswalk, get the HUC10 containing the above gage
cross = pd.read_parquet(attribute_paths['usgs_huc_crosswalk'])
huc12_id = cross.loc[cross['primary_location_id']==usgs_id, 'secondary_location_id'].iloc[0]
huc10_id = "-".join(['huc10', huc12_id.split("-")[1][:10]])

pcp_df = du.run_teehr_query(
    query_type="timeseries",
    scenario=MRF_forcing,
    location_id=huc10_id,
    value_time_start=widgets['value_time'][1].value_start,    
    value_time_end=widgets['value_time'][1].value_end,    
    reference_time_start=widgets['reference_time'][1].value_start,    
    reference_time_end=widgets['reference_time'][1].value_end,
    attribute_paths=attribute_paths,
    return_query=False,
    include_geometry=False,
)
display(pcp_df.head())

Now let's create linked precipitation and streamflow time series overlay plots

In [ ]:
pcp_df = pcp_df.rename(columns={'primary_value':'obs_pcp','secondary_value':'fcst_pcp'})
pcp_obs = pcp_df.hvplot(x='value_time', y='obs_pcp', label='observed', ylabel='Precip (mm)', color = 'black', line_width=3)   
hyetograph = pcp_obs
for t, time in enumerate(ref_times):
    df_t = pcp_df[pcp_df['reference_time'] == time]
    ci = cmap[cstep * t]
    fcst = df_t.hvplot(x='value_time', y='fcst_pcp', color = [ci])
    hyetograph = hyetograph * fcst

((hyetograph * pcp_obs) + (hydrographs * flow_obs)).cols(1)